In [2]:
import json
import os
import time
import parmap

import nltk
import numpy as np
from konlpy.tag import Okt
from khaiii import KhaiiiApi
from tensorflow.keras import models, layers, optimizers, losses, metrics
import parmap

In [3]:
def read_data(filename):
    with open(filename, 'r') as f:
        data = [line.split('\t') for line in f.read().splitlines()]     
        # txt 파일의 헤더(id document label)는 제외하기
        data = data[1:]
    return data

def tokenizing(docs):
    return ['/'.join(t) for t in okt.pos(docs,norm=True, stem=True)]

def tokenizing_par(row):
    return tokenizing(row[1]), row[2]

def term_frequency(doc):
    return [doc.count(word) for word in selected_words]

In [4]:
okt = Okt()
okt.pos('순천향대학교 컴퓨터공학과 16학번 송희령') #테스트

[('순천향대', 'Noun'),
 ('학교', 'Noun'),
 ('컴퓨터공학', 'Noun'),
 ('과', 'Josa'),
 ('16', 'Number'),
 ('학번', 'Noun'),
 ('송희령', 'Noun')]

In [5]:
train_df = read_data('ratings_train.txt')
test_df = read_data('ratings_test.txt')

In [6]:
print(len(test_df))

50000


In [7]:
train_pos = [] #훈련데이터
test_pos = [] #테스트 데이터
for row in train_df:
    try:
        train_pos0 = [tokenizing(row[1]),row[2]] 
        #리스트 안에 한문장에 대해서 위에서 만든 tokenizing함수를 통해서 [[토큰화텍스트],긍/부정 여부]를
        #리스트의 각문장별로 요소로 넣는다.
        train_pos.append(train_pos0)
    except:
        pass
    
for row in test_df:
    try:
        test_pos0 = [tokenizing(row[1]),row[2]] 
        test_pos.append(test_pos0)
    except:
        pass

In [8]:
tokens = [t for d in train_pos for t in d[0]]

In [9]:
print(len(tokens))

2159921


In [10]:
text = nltk.Text(tokens,name='NMSC')

In [11]:
print(text.vocab())

<FreqDist with 49895 samples and 2159921 outcomes>


In [12]:
nWords = 10000

In [13]:
selected_words = [f[0] for f in text.vocab().most_common(nWords)]

In [14]:
train_x = [term_frequency(d) for d, _ in train_pos]
test_x = [term_frequency(d) for d, _ in test_pos]
train_y = [c for _, c in train_pos]
test_y = [c for _, c in test_pos]

In [15]:
x_train = np.asarray(train_x).astype('float32')
x_test = np.asarray(test_x).astype('float32')

y_train = np.asarray(train_y).astype('float32')
y_test = np.asarray(test_y).astype('float32')

In [16]:
y_train

array([0., 1., 0., ..., 0., 1., 0.], dtype=float32)

In [17]:
model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(nWords,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [18]:
model.compile(optimizer=optimizers.RMSprop(lr=0.001), loss=losses.binary_crossentropy, metrics=[metrics.binary_accuracy])

In [ ]:
model.fit(x_train, y_train, epochs=10, batch_size=512)
results = model.evaluate(x_test, y_test)

In [20]:
results

[0.41163212060928345, 0.8483399748802185]

In [21]:
def ppn(review):
    token = tokenizing(review)
    tf = term_frequency(token)
    data = np.expand_dims(np.asarray(tf).astype('float32'), axis=0)
    score = float(model.predict(data))
    if(score > 0.5):
        print("[{}]는 {:.2f}% 확률로 긍정 리뷰이지 않을까 추측해봅니다.^^\n".format(review, score * 100))
    else:
        print("[{}]는 {:.2f}% 확률로 부정 리뷰이지 않을까 추측해봅니다.^^;\n".format(review, (1 - score) * 100))

In [22]:
ppn("주연배우 때문에 봤어요")

[주연배우 때문에 봤어요]는 62.81% 확률로 부정 리뷰이지 않을까 추측해봅니다.^^;



In [23]:
ppn("이게 지금 정부에서 할 짓이냐.구멍가게도 아니고. 무능력한 것들이 정권을 잡으니 완전 개판이다.")

[이게 지금 정부에서 할 짓이냐.구멍가게도 아니고. 무능력한 것들이 정권을 잡으니 완전 개판이다.]는 98.99% 확률로 부정 리뷰이지 않을까 추측해봅니다.^^;



In [24]:
ppn("우리나라 정치인들이 보고 배워야 할 의인들♡")

[우리나라 정치인들이 보고 배워야 할 의인들♡]는 95.99% 확률로 긍정 리뷰이지 않을까 추측해봅니다.^^



In [25]:
ppn("믿고 보는 감독이지만 이번에는 아니네요")

[믿고 보는 감독이지만 이번에는 아니네요]는 55.69% 확률로 긍정 리뷰이지 않을까 추측해봅니다.^^



In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                640064    
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 644,289
Trainable params: 644,289
Non-trainable params: 0
_________________________________________________________________


In [27]:
results

[0.41163212060928345, 0.8483399748802185]

In [28]:
ppn("20164091 송희령")

[20164091 송희령]는 53.55% 확률로 긍정 리뷰이지 않을까 추측해봅니다.^^



In [29]:
ppn("싫다 좋다 싫다 싫다 좋다")

[싫다 좋다 싫다 싫다 좋다]는 79.18% 확률로 부정 리뷰이지 않을까 추측해봅니다.^^;



In [30]:
ppn("좋다 싫다 좋다 좋다 싫다")

[좋다 싫다 좋다 좋다 싫다]는 52.58% 확률로 부정 리뷰이지 않을까 추측해봅니다.^^;



In [31]:
ppn("좋다 좋다 좋다 좋다 좋다")

[좋다 좋다 좋다 좋다 좋다]는 93.16% 확률로 긍정 리뷰이지 않을까 추측해봅니다.^^



In [ ]:
ppn("싫다 싫다 싫다 싫다 싫다")

In [ ]:
ppn("의인")

In [ ]:
ppn("좋다")

In [ ]:
ppn("싫다")

In [ ]:
ppn("잘 못 만든 영화")

In [ ]:
ppn("잘 만든 영화")

In [ ]:
ppn("더빙이 좋네요")

In [ ]:
ppn("dell")

In [ ]:
ppn("
액션도 쩔고 계속 장소가 바뀌며 치달리는 영화라 지루하지도 않다! 황정민 이정재 박정민 연기 구멍이 없어서 더 좋은듯")

In [ ]:
ppn("액션도 쩔고 계속 장소가 바뀌며 치달리는 영화라 지루하지도 않다! 황정민 이정재 박정민 연기 구멍이 없어서 더 좋은듯")

In [ ]:
ppn("정신병걸리는 영화였음 뭔 되도안한 설정으로 서로 죽이려고 사람이 산다고? 태어나서 첨으로 평점달아본다 진짜로 ㅋ")

In [ ]:
ppn("화려하지만 개연성이 전혀 없는 스토리")

In [ ]:
ppn(" 저는 평점에 속았습니다....")

In [ ]:
ppn("쟤네가 걸리고 싶어서 걸렸겠냐 ㅠㅠㅠ")

In [ ]:
ppn("서울시 주최 오페라관람 8500명은 전혀 검사도 안하고, 박원순 서울시장 장례식에 모여든 수만명, 해수욕장 인파 80 만명은 전혀 코로나랑 상관없다는 이야기냐? ,이렇게 되도록 그동안 방역불감증을 자초한 정부. 없는 휴일도 만들어내고, 외식 극장 문화쿠폰 발행까지 했던건 잘못이 아니냐?")

In [ ]:
ppn("이렇게 라이브인증을 하다니...에코 1도 없는 쌩목라이브에 댄스완벽..미친그룹이다 진짜 ㄷㄷㄷ")